In [1]:
IMG_SIZE = 512
BATCH_SIZE = 16
EPOCHS = 5
VAL_SPLIT = 0.2

# Data Loading

In [2]:
import tensorflow as tf

dataset_path = "../../data-collection/image-backend/saved_images"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    class_names=['hazard', 'non-hazard'],
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=42,
    validation_split=VAL_SPLIT,
    subset="training",
)

Found 160 files belonging to 2 classes.
Using 128 files for training.


In [3]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels='inferred',
    label_mode='int',
    class_names=['hazard', 'non-hazard'],
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=VAL_SPLIT,
    subset="validation",
)

Found 160 files belonging to 2 classes.
Using 32 files for validation.


In [4]:
train_dataset.class_names

['hazard', 'non-hazard']

# Model Fine-tuning

In [5]:
def build_model():
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
    )

    base_model.trainable = False

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)

    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )
    
    return model

In [6]:
model = build_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 512, 512)  │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 512, 512)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 512, 512)  │          0 │ input_layer_1[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 512, 512,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 512, 512,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 16, 16,    │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2048)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │      2,049 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
model.fit(train_dataset, epochs=EPOCHS, verbose=2, validation_data=validation_dataset)

Epoch 1/5
8/8 - 31s - 4s/step - binary_accuracy: 0.6250 - loss: 0.6703 - val_binary_accuracy: 0.5938 - val_loss: 0.5649
Epoch 2/5
8/8 - 27s - 3s/step - binary_accuracy: 0.6172 - loss: 0.5852 - val_binary_accuracy: 0.7188 - val_loss: 0.4952
Epoch 3/5
8/8 - 30s - 4s/step - binary_accuracy: 0.7031 - loss: 0.5207 - val_binary_accuracy: 0.7500 - val_loss: 0.4614
Epoch 4/5
8/8 - 30s - 4s/step - binary_accuracy: 0.7656 - loss: 0.4652 - val_binary_accuracy: 0.8125 - val_loss: 0.4276
Epoch 5/5
8/8 - 31s - 4s/step - binary_accuracy: 0.7969 - loss: 0.4237 - val_binary_accuracy: 0.8125 - val_loss: 0.4045


In [8]:
model.save("../models/ResNet.keras")